In [22]:
import sys
sys.path.append('../../nucleon_elastic_FF/scripts/area51_files')
sys.path.append('../../nucleon_elastic_FF/scripts')
import sources
import utils
from lattedb.wavefunction.models import Hadron as wavefunction_Hadron
from lattedb.wavefunction.models import Hadron4D as wavefunction_Hadron4D
from lattedb.fermionaction.models import Hisq as fermionaction_Hisq
from lattedb.gaugeaction.models import LuescherWeisz as gaugeaction_LuescherWeisz
from lattedb.fermionaction.models import MobiusDW as fermionaction_MobiusDW
from lattedb.gaugeconfig.models import Nf211 as gaugeconfig_Nf211
from lattedb.quarksmear.models import Point as quarksmear_Point
from lattedb.linksmear.models import WilsonFlow as linksmear_WilsonFlow
from lattedb.propagator.models import BaryonCoherentSeq as propagator_BaryonCoherentSeq
from lattedb.propagator.models import OneToAll as propagator_OneToAll

In [23]:
# change parameters starting here
short_tag = "a09m310"
stream = "e"
gaugeconfigs = gaugeconfig_Nf211.objects.filter(short_tag=short_tag, stream=stream)
import a09m310 as a51

In [24]:
p = a51.mpirun_params("summit")
p["STREAM"] = stream
cfgs_run, p['srcs'] = utils.parse_cfg_src_argument([p["cfg_i"], p["cfg_f"], p["cfg_d"]],'',p)

In [21]:
# sink smear of sequential propagator
# this should always be point
sinksmear, created = quarksmear_Point.objects.get_or_create(
	tag="point", # (Optional) User defined tag for easy searches
	description="Point", # (Optional) Description of the quark smearing operator
)

linksmear, created = linksmear_WilsonFlow.objects.get_or_create(
   	flowtime=p["FLOW_TIME"], # Flow time in lattice units
	flowstep=p["FLOW_STEP"], # Number of diffusion steps
)

for quark_tag in ["up", "down"]:
    fermion_tag = f"M5{p['M5']}_L5{p['L5']}_a{p['alpha5']}"
    fermionaction, created = fermionaction_MobiusDW.objects.get_or_create(
        quark_mass=p["MV_L"], # Input quark mass
        quark_tag=quark_tag, # Type of quark
        l5=p["L5"], # Length of 5th dimension
        m5=p["M5"], # 5th dimensional mass
        b5=p["B5"], # Mobius kernel parameter [a5 = b5 - c5, alpha5 * a5 =…
        c5=p["C5"], # Mobius kernal perameter
        linksmear=linksmear, # Foreign Key pointing to additional gauge `linksmear` outside of Monte Carlo.
        tag=fermion_tag, # (Optional) User defined tag for easy searches
    )
    if quark_tag == "up": qtag = "UU"
    if quark_tag == "down": qtag = "DD"
    for parity in [1, -1]:
        for spin_z_x2 in [1, -1]:
            if parity == 1: paritytag = "pp"
            else: paritytag = "np"
            if spin_z_x2 == 1: spintag = "up"
            else: spintag = "dn"
            sinkwave, created = wavefunction_Hadron.objects.get_or_create(
                strangeness=0, # Strangeness of hadronic operator
                irrep="G", # Irreducible representations of O^D_h (octahedral group)
                embedding=1, # k-th embedding of O^D_h irrep., can be blank
                parity=parity, # Parity of hadronic operator
                spin_x2=1, # Total spin times 2
                spin_z_x2=spin_z_x2, # Spin in \(z\)-direction
                isospin_x2=1, # Total isospin times 2
                isospin_z_x2=1, # Isospin in \(z\)-direction times 2
                momentum=0, # Momentum in units of 2 pi / L
                tag="proton", # (Optional) User defined tag for easy searches
                description=f"G1 irrep {paritytag} spin {spintag} proton", # (Optional) Description of the interpolating operator
            )
            if spin_z_x2 == 1: snkspntag = "up"
            else: snkspntag = "dn"
            sourcewave, created = wavefunction_Hadron4D.objects.get_or_create(
                strangeness=0, # Strangeness of hadronic operator
                irrep="G", # Irreducible representations of O^D_h (octahedral group)
                embedding=1, # k-th embedding of O^D_h irrep., can be blank
                parity=parity, # Parity of hadronic operator
                spin_x2=1, # Total spin times 2
                spin_z_x2=spin_z_x2, # Spin in \(z\)-direction
                isospin_x2=1, # Total isospin times 2
                isospin_z_x2=1, # Isospin in \(z\)-direction times 2
                tag="proton", # (Optional) User defined tag for easy searches
                description=f"G1 irrep {paritytag} spin {spintag} proton", # (Optional) Description of the interpolating operator
            )
            if spin_z_x2 == 1: srcspntag = "up"
            else: srcspntag = "dn"
            for gaugeconfig in gaugeconfigs:
                for sinksmear_bool in [False, True]:
                    propagators = propagator_OneToAll.objects.filter(gaugeconfig=gaugeconfig,
                                                                     fermionaction__mobiusdw__quark_tag="light",
                                                                     sinksmear__point__isnull=sinksmear_bool,
                                                                    )
                    if propagators[0].sourcesmear.type == "Point": srcsmrtag = "P"
                    else: srcsmrtag = "S"
                    if propagators[0].sinksmear.type == "Point": snksmrtag = "P"
                    else: snksmrtag = "S"
                    smrtag = f"{snksmrtag}{srcsmrtag}"
                    for sinksep in p["t_seps"]:
                        mq = propagators[0].tag.split('_')[2][2:]
                        paction = f"{propagators[0].tag.split('wflow')[1].split('_cfg')[0]}_{propagators[0].tag.split('wv_')[1]}"
                        pmom = f"px{sinkwave.momentum}py{sinkwave.momentum}pz{sinkwave.momentum}"
                        tag = f"seqprop_{gaugeconfig.short_tag}_{gaugeconfig.stream}_{gaugeconfig.config}_{sinkwave.tag}_{paritytag}_{qtag}_{snkspntag}_{srcspntag}_gf{paction}_mq{mq}_{pmom}_dt{parity*sinksep}_Srcs0-7_{smrtag}"
                        propagator_baryoncoherentseq, created = propagator_BaryonCoherentSeq.objects.get_or_create(
                            gaugeconfig=gaugeconfig, # Foreign Key referencing specific `gaugeconfig` inverted on
                            fermionaction=fermionaction, # Foreign Key referencing valence lattice `fermionaction`
                            sourcewave=sourcewave,
                            sinkwave=sinkwave, # Foreign Key referencing sink interpolating operator `wavefunction`
                            sinksmear=sinksmear, # Foreign Key pointing to sink `quarksmear` which should be Point unless some…
                            sinksep=sinksep, # Source-sink separation time
                            tag=tag# (Optional) User defined tag for easy searches
                        )
                        propagator_baryoncoherentseq.propagator0.add(*propagators)
                        propagator_baryoncoherentseq.propagator1.add(*propagators)
                        print(f"\r>>{created} {tag}", end="")

>>True seqprop_a09m310_e_306_proton_np_DD_dn_dn_gf1.0_n40_M51.1_L56_a1.5_w3.5_n45_mq0.00951_px0py0pz0_dt-14_Srcs0-7_SS